<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Иван, ты хорошо доработал проект и получил интересные результаты, надеюсь ты прокачал свои навыки и узнал много нового! Желаю тебе успехов в дальнейшем постижении анализа данных.
    
P.s. только больше не удаляй комментарии ревьюверов

</div>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## План исследования

### 1. Общая информация о данных
    - Посмотрим начало и конец датасета, отдельные столбцы
    - Сделаем предварительные выводы
    - Поищем аномалии
### 2. Предобработка данных
    - Определим и заполним пропущенные значения
    - Заменим вещественный тип данных на целочисленный
    - Удалим дубликаты
    - Выделим леммы в значениях столбца с целями получения кредита
    - Категоризируем данные
### 3. Ответ на поставленные вопросы
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
### 4. Результаты исследования
    - Общий вывод
    - Рекомендации

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [2]:
ks = pd.read_csv('/datasets/data.csv')

## Описание данных
    •	children — количество детей в семье
    •	days_employed — общий трудовой стаж в днях
    •	dob_years — возраст клиента в годах
    •	education — уровень образования клиента
    •	education_id — идентификатор уровня образования
    •	family_status — семейное положение
    •	family_status_id — идентификатор семейного положения
    •	gender — пол клиента
    •	income_type — тип занятости
    •	debt — имел ли задолженность по возврату кредитов
    •	total_income — ежемесячный доход
    •	purpose — цель получения кредита


In [3]:
ks.info() # Общая информация по датасету

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
ks.describe() # Общие параметры таблицы

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


#### Посмотрим начало и конец таблицы

In [5]:
ks.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [6]:
ks.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


#### Посмотрим на отдельные столбцы таблицы для выявления аномальных значений

In [7]:
ks['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [8]:
ks['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [9]:
ks['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [10]:
ks['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [11]:
ks['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
ks['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [13]:
ks['debt'].unique()

array([0, 1])

In [14]:
ks['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Вывод

    - Датасет состоит из 21525 строк.
    - В столбцах days_employed, total_income есть 2 174 пропущенных значений. Можно сделать предположение, что эти клиенты не предоставили информацию о своём месте работе и доходах, т.е работают не официально или вообще не работают. Такого рода клиенты относятся к категории высокого риска для банка. Так как в рамках данного исследования столбец total_income нужен для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?", то необходимо обработать эти пропуски.
    - Отмечаем, что столбцы days_employed, total_income в формате float. Для удобства в дальнейшем переведем их в тип Int.
    - Столбцец children: аномальные значение -1 и 20. -1 заменим на 1, 20 на 2 - предположительно ошибка в данных.
    - Столбец days_employed: Отрицательные значения - переведем в положительные. Максимальное значение в столбце соответствует 1100 лет работы, что явно являтеся ошибкой в данных, но на текущее исследование не влияет.
    - Стобец dob_years: У 101 одного человека возраст 0. Возьмём на заметку, в данном случае на вопросы исследования не влияет.
    - Стобец education: Есть записи в верхнем регистре - переведём в нижний.
    - Столбец family_status: Есть записи в верхнем регистре - переведём в нижний.
    - Стобец gender: 1 клиент с неизвестным полом. На исследование не влияет.
    - Столбец total_income: Переведем в целочисленный тип данных. Посмотрим на предмет аномальных значений.  
    - Столбец purpose: Есть разные формулировки одних и тех же целей кредита - применим лемматизацию.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец, датафрейм осмотрел и сформулировал подходящие выводы

</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [15]:
# Подсчитаем суммарное количество пропусков
ks.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Есть одинаковое колличество пропущенных значений в двух столбцах.
Посмотрим подробнее на характер пропущенных значений.

In [16]:
ks[ks['total_income'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [17]:
ks[ks['total_income'].isnull()].groupby('income_type')['debt'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: debt, dtype: int64

In [18]:
ks[ks['total_income'].isnull()].groupby('education')['debt'].count()

education
ВЫСШЕЕ                   23
Высшее                   25
НАЧАЛЬНОЕ                 1
НЕОКОНЧЕННОЕ ВЫСШЕЕ       7
Начальное                 1
Неоконченное высшее       7
СРЕДНЕЕ                  67
Среднее                  65
высшее                  496
начальное                19
неоконченное высшее      55
среднее                1408
Name: debt, dtype: int64

In [19]:
ks[ks['total_income'].isnull()].groupby('dob_years')['debt'].count()

dob_years
0     10
19     1
20     5
21    18
22    17
23    36
24    21
25    23
26    35
27    36
28    57
29    50
30    58
31    65
32    37
33    51
34    69
35    64
36    63
37    53
38    54
39    51
40    66
41    59
42    65
43    50
44    44
45    50
46    48
47    59
48    46
49    50
50    51
51    50
52    53
53    44
54    55
55    48
56    54
57    56
58    56
59    34
60    39
61    38
62    38
63    29
64    37
65    20
66    20
67    16
68     9
69     5
70     3
71     5
72     2
73     1
Name: debt, dtype: int64

Видим, что пропущены значения разных категорий заёмщиков. Скорее всего пропуски имеют технический характер.  
Вариант с тем, что заёмщики вообще не работали или не предоставили документы о трудовом стаже и доходах маловероятен, так как в этом случае должна быть отдельная отметка в данных или модель построения кредитного скорринга.   
Всего пропущено около **10%** значений - оптимальным решением было бы запросить информацию о причинах пропущенных значений у заказчика.  
В рамках исследования столбец total_income нужен для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"  
Мы не можем однозначно установить причину, поэтому есть 2 варианта решения проблемы:  
1) Заменить пропущенные значение на "0" и не учитывать их при ответе на вопрос.  
2) Заменить на медианные значения в зависимости от уровня образования и типа занятости.  
Воспользуемся 2 вариантом так как более вероятна техническая ошибка и терять 10% датасета нецелесообразно.   
Для ответа на остальные вопросы столбцы days_employed, total_income не требуются.

In [20]:
# Найдём медианные значения по столбцам 'education_id','income_type' через группировку и составим "словарь"
median_gr_ti = ks.groupby(['education_id','income_type'])['total_income'].median()
median_gr_ti

education_id  income_type    
0             безработный        202722.511368
              госслужащий        172511.107016
              компаньон          201785.400018
              пенсионер          144240.768611
              предприниматель    499163.144947
              сотрудник          165640.744634
              студент             98201.625314
1             безработный         59956.991984
              в декрете           53829.130729
              госслужащий        136652.970357
              компаньон          159070.690289
              пенсионер          114842.854099
              сотрудник          136555.108821
2             госслужащий        160592.345303
              компаньон          179867.152890
              пенсионер          120136.896353
              сотрудник          151308.937846
3             госслужащий        148339.290825
              компаньон          136798.905143
              пенсионер          102598.653164
              сотрудник       

In [21]:
# Составим функцию для возврата медианного значения из словаря
def fill_total_income(row):
    return median_gr_ti.loc[row['education_id']][row['income_type']]

In [22]:
# Вызываем функцию, на вход передаём строки из таблицы с нулевыми значениями в столбец total_income
ks.loc[ks['total_income'].isnull(),'total_income'] = ks.apply(fill_total_income, axis=1)

In [23]:
# Для исследования не требуется, но проделаем теже действия со столбцом days_employed
median_gr_de = ks.groupby(['education_id','income_type'])['days_employed'].median()
median_gr_de

education_id  income_type    
0             безработный        395302.838654
              госслужащий         -2531.034209
              компаньон           -1454.659104
              пенсионер          366158.526428
              предприниматель      -520.848083
              сотрудник           -1556.457266
              студент              -578.751554
1             безработный        337524.466835
              в декрете           -3296.759962
              госслужащий         -2857.770974
              компаньон           -1670.809068
              пенсионер          365025.338867
              сотрудник           -1613.581420
2             госслужащий         -1885.183639
              компаньон           -1017.888120
              пенсионер          372250.501660
              сотрудник           -1197.676879
3             госслужащий         -2787.767403
              компаньон           -1151.634460
              пенсионер          360264.985350
              сотрудник       

In [24]:
def fill_days_employed(row):
    return median_gr_de.loc[row['education_id']][row['income_type']]

In [25]:
ks.loc[ks['days_employed'].isnull(),'days_employed'] = ks.apply(fill_days_employed, axis=1)

In [26]:
# Все нулевые значения заменены
ks.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец, хороший подход для заполнения пропусков!
    

</div>

In [27]:
# Проверим выведя первую строку, которая была с нулями.
ks.loc[12]

children                           0
days_employed                 365025
dob_years                         65
education                    среднее
education_id                       1
family_status       гражданский брак
family_status_id                   1
gender                             M
income_type                пенсионер
debt                               0
total_income                  114843
purpose              сыграть свадьбу
Name: 12, dtype: object

Теперь посмотрим на предмет аномальных значений.

In [28]:
# Проверим столбец на предмет "аномальных" значений.
ks.groupby('income_type')['total_income'].agg(['count','median','mean','max','min'])

,count,median,mean,max,min
income_type,,,,,
безработный,2,131339.751676,131339.751676,2.027225e+05,59956.991984
в декрете,1,53829.130729,53829.130729,5.382913e+04,53829.130729
госслужащий,1459,148978.800029,169086.211531,9.104515e+05,29200.077193
компаньон,5085,170155.716801,199743.688235,2.265604e+06,28702.812889
пенсионер,3856,114842.854099,135164.404380,7.351033e+05,20667.263793
предприниматель,2,499163.144947,499163.144947,4.991631e+05,499163.144947
сотрудник,11119,138068.631966,159568.189866,1.726276e+06,21367.648356
студент,1,98201.625314,98201.625314,9.820163e+04,98201.625314


Есть достаточно существенная разница между средним и медианой, что говорит об аномалиях.

In [29]:
# Посмотри на этих клиентов
ks[(ks['total_income'] > 500000)].sort_values('total_income',ascending=False).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
17503,0,-2285.476482,43,среднее,1,женат / замужем,0,M,компаньон,0,1.597613e+06,операции с недвижимостью
18368,1,-333.935516,41,ВЫСШЕЕ,0,гражданский брак,1,M,компаньон,0,1.551153e+06,свадьба
18353,1,-3173.282035,41,высшее,0,Не женат / не замужем,4,F,компаньон,0,1.427934e+06,автомобиль
15268,1,-10207.448165,64,высшее,0,в разводе,3,M,компаньон,0,1.350246e+06,жилье
11071,1,-1851.200013,36,высшее,0,гражданский брак,1,F,сотрудник,0,1.286281e+06,покупка коммерческой недвижимости


In [30]:
# Посмотрим на колличество клиентов с доходом свыше 500т.руб в месяц
ks[(ks['total_income'] > 500000)].count()

children            222
days_employed       222
dob_years           222
education           222
education_id        222
family_status       222
family_status_id    222
gender              222
income_type         222
debt                222
total_income        222
purpose             222
dtype: int64

In [31]:
z=ks[(ks['total_income'] > 500000)]
z.groupby('income_type')['debt'].mean()

income_type
госслужащий    0.000000
компаньон      0.068966
пенсионер      0.000000
сотрудник      0.082192
Name: debt, dtype: float64

Весьма вероятно сверхвысокие показатели дохода также являются технической ошибкой, возможно не там стоит запятая.
Также как с пропусками однозначно утверждать на основании имеющихся данных мы не можем.  
Таких значений около **1%**, поэтому оставим их как есть.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 
    
Госслужащий, компаньон и сотрудник вполне могут зарабатывать больше 500

</div>

### Вывод

Мы избавились от пропусков, оставив только необходимые для будущего анализа данные.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец, но:
    
* стоило поработать с нулевым возрастом, это маркер неявного пропуска

* значением XNA по полу

</div>

<font color='blue'> Отлично - избавились от пропусков.</font>

### Замена типа данных

Производим замену типа данных на целочисленный, а также обработаем аномальные значение и проблемы с регистром.

In [32]:
# Проверим колличество дублей до обработки данных
ks.duplicated().sum()

54

In [33]:
# В столбце children заменим значения -1 на 1, 20 на 2. Явно ошибка в данных. 
# В реальности такого плана ошибки в данных заворачиваются на этапе верификации анкеты.
ks['children'] = ks['children'].replace(20, 2)
ks['children'] = ks['children'].replace(-1, 1)
ks['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [34]:
# Столбцец days_employed не требуется для ответа на вопросы, но для порядка его также обработаетм.
# Избавляемся от отрицательных значений и переводим в целочисленный тип.
# Используем метод astype(), так как нам нужен формат int.
# Погрешность при переводе не существенна для исследования.
ks['days_employed'] = ks['days_employed'].astype('int').abs()
ks['days_employed'].describe()

count     21525.000000
mean      67299.835726
std      139401.710546
min          24.000000
25%        1023.000000
50%        1989.000000
75%        5347.000000
max      401755.000000
Name: days_employed, dtype: float64

Как уже отмечали ранее максимальное значение 401775/365=1100 лет. В рамках данного исследования не обращаем внимание.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Как вариант, можно было создать новый признак года, куда вносить значение в годах, а если оно превышает возраст клиента, то значение возраст - 18 лет

</div>

In [35]:
# Столбец education: Есть записи в верхнем регистре - переведём в нижний.
ks['education'] = ks['education'].str.lower()
ks['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [36]:
# Столбец family_status: Есть запись "Не женат" в верхнем регистре - переведём в нижний.
ks['family_status'] = ks['family_status'].str.lower()
ks['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [37]:
# Столбец total_income: Переведем в целочисленный тип данных.
ks['total_income'] = ks['total_income'].astype('int')
ks['total_income'].head(5)

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

In [38]:
ks.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [39]:
ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Данные обработаны и готовы для дальнейшего исследования.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец!
</div>

### Обработка дубликатов

In [40]:
# Проверим колличество дубликатов
ks.duplicated().sum()

71

Колличество дубликатов **71**, что составляет менее 1% от общего колличества.  
17 из них появились после перевода в нижний регистр, изменения типов данных и обработки аномалий.

In [41]:
# Удалим дубли и сбросим индекс
ks = ks.drop_duplicates().reset_index(drop=True)
ks.duplicated().sum()

0

### Вывод

Дубликаты удалены.  
Можно предположить, что причина появления связана с повторной подачей заявок на кредит.  
Также есть небольшая вероятность того, что были удалены данные разных людей так как у нас нет например ФИО клиента для идентификации клиента.  

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Отлично, от дубликатов избавился)

</div>

### Лемматизация

In [42]:
# Загрузим необходмую библиотеку
from pymystem3 import Mystem
m = Mystem()

In [43]:
# Посмотрим на уникальные значения столбца purpose
unique_credit_purposes = ks['purpose'].unique()
print(unique_credit_purposes)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [44]:
# Проходим по столбцу purpose, выделяем леммы и делаем подсчёт уникальных лемматизированных слов.
from collections import Counter
lem_purposes = []
for p in ks['purpose']:
    lem_purposes +=m.lemmatize(p)
print(Counter(lem_purposes)) 

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


In [45]:
# Сохраним основные леммы целей получение кредита.
purpose_cat = [
    'свадьба',
    'недвижимость',
    'жилье',
    'автомобиль',
    'образование'
]

### Вывод

Лемматизация целей получения кредита проведена.  
Если отбросить пробелы и прочие общие слова, можно выделить следующие леммы в получение кредита:  
    - недвижимость
    - жилье
    - автомобиль
    - образование
    - свадьба

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец, лемматризацию провел хорошо, наиболее популярные цели выделены верно

</div>

### Категоризация данных

##### Для ответа на поставленные вопросы проведём дополнительную категоризацию данных

**•	Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [46]:
# Посмотрим ещё раз на колличество клиентов в каждой категории
ks['children'].value_counts().sort_index()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Видим, что больше всего клиентов у которых нет детей.
Разделим клиентов на 2 категории:  
**"Нет детей"**  
**"Есть дети"**

In [47]:
# Напишем соответсвующую функцию
def kids_group(count):
    if count == 0:
        return 'Нет детей'
    return 'Есть дети'
# Протестируем
print(kids_group(0))
print(kids_group(1))

Нет детей
Есть дети


In [48]:
# Добавим в таблицу новый столбец
ks['kids'] = ks['children'].apply(kids_group)

In [49]:
ks.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,kids
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Нет детей


**•	Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [50]:
ks['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Будем исходить из логики, что клиенты "женат / замужем, гражданский брак" будут выплачивать кредит совместно с созаемищиком.  
Клиенты не женат / не замужем,в разводе,вдовец / вдова в одиночку.  
Категоризируем клиентов на 2 категории:  
 - В браке
 - Не в браке

In [51]:
# Напишем соответсвующую функцию
def cat_famstat(categ):
    if (categ == 'женат / замужем') or (categ == 'гражданский брак'):
        return 'В браке'
    return 'Не в браке'
# Протестируем
print(cat_famstat('женат / замужем'))
print(cat_famstat('вдовец / вдова'))

В браке
Не в браке


In [52]:
# Добавим в таблицу новый столбец
ks['family_status_category'] = ks['family_status'].apply(cat_famstat)

In [53]:
ks.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,kids,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Есть дети,В браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Есть дети,В браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Нет детей,В браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Есть дети,В браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Нет детей,В браке


**•	Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Категоризируем клиентов на 4 категории в зависимости от уровня дохода.  
Разобъём на квантили.

In [54]:
ks['total_income'].quantile([0.25,0.5,0.75])

0.25    107515.5
0.50    143707.5
0.75    198307.0
Name: total_income, dtype: float64

In [55]:
# Напишем функцию. Клиентам с неизвестным доходом присвоим значение "Уровень дохода неизвестен"
def income_status(total_income):
    if total_income <= 107515.5:
            return 'Низкий уровень дохода'
    elif total_income <= 143707.5:
            return 'Средний уровень дохода'
    elif total_income <= 198307.0:
            return 'Высокий уровень дохода'      
    return 'Сверхвысокий уровень дохода'

In [56]:
ks['income_level'] = ks['total_income'].apply(income_status)

In [57]:
ks.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,kids,family_status_category,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Есть дети,В браке,Сверхвысокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Есть дети,В браке,Средний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Нет детей,В браке,Высокий уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Есть дети,В браке,Сверхвысокий уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Нет детей,В браке,Высокий уровень дохода


In [58]:
ks.groupby(['income_level'])['total_income'].agg(['count','median'])

,count,median
income_level,,
Высокий уровень дохода,5363,165640.0
Низкий уровень дохода,5364,83881.0
Сверхвысокий уровень дохода,5364,254276.5
Средний уровень дохода,5363,127955.0


**•	Как разные цели кредита влияют на его возврат в срок?**

Ранее мы уже выделили леммы в целях кредита. Осталось добавить соответствующий столбец.  

In [59]:
# Выведем список лемм
purpose_cat

['свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование']

В наших данных слова встречаются в разном виде, например "покупка жилья", "операции с жильем".  
Можно отбросить окончания, но более правильным будет написать функцию, которая будет лемматизировать строку и сравнивать с нашим списком лемм.

In [60]:
# Пишем функцию, которая возвращает после проверки лемму из списка.
def cat_purpose(text):
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        for category in purpose_cat:
            if category in lemma:
                return category

# Протестируем функцию
cat_purpose('покупка жилья')

'жилье'

In [61]:
ks['purpose_cat'] = ks['purpose'].apply(cat_purpose)

In [62]:
ks.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,kids,family_status_category,income_level,purpose_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Есть дети,В браке,Сверхвысокий уровень дохода,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Есть дети,В браке,Средний уровень дохода,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Нет детей,В браке,Высокий уровень дохода,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Есть дети,В браке,Сверхвысокий уровень дохода,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Нет детей,В браке,Высокий уровень дохода,свадьба


In [63]:
ks['purpose_cat'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_cat, dtype: int64

### Вывод

Мы провели категоризацию данных.  
В таблице есть все метрики необходимые для ответа на вопросы заказчика.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец, у тебя хорошие и интересные способы категоризации!

</div>

### Шаг 3. Ответьте на вопросы

Для начала посмотрим на общий процент задолженностей по кредиту.

In [64]:
ks['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Всего у нас **21454** клиента.

In [65]:
1741/21454*100

8.115036822970076

**8.1%** клиентов имели задолженность по кредитам.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец, что подсчитал, будем сравнивать с этим порогом

</div>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [66]:
# Сгруппируем данные и посмотрим на процент задолженностей по категориям.  
# Выделим цветом, категории где процент выше общего среднего.
(ks.groupby(['kids'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,count,mean
kids,,
Есть дети,7363,9.2%
Нет детей,14091,7.5%


In [67]:
# Посмотрим также в разрезе колличества детей.
(ks.groupby(['children'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,count,mean
children,,
4,41,9.8%
2,2128,9.5%
1,4855,9.2%
3,330,8.2%
0,14091,7.5%
5,9,0.0%


### Вывод

Наличие детей влияет на наличие задолженностей по кредиту.
- **9.2%** клиентов имели задолженность по кредиту, что на 1% больше среднего показателя.
  Выбивается показатель для семей с 3-мя детьми, но т.к выборка по ним мала, делать какие-то выводы нецелесообразно.  
- Заёмщики без детей реже залезают в долги, всего **7.2%**

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Отлично! Как мы видим дети существенно влияют на возврат кредита
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [68]:
# Сгруппируем данные
(ks.groupby(['family_status_category'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,count,mean
family_status_category,,
Не в браке,4964,8.5%
В браке,16490,8.0%


In [69]:
(ks.groupby(['family_status'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,count,mean
family_status,,
не женат / не замужем,2810,9.8%
гражданский брак,4151,9.3%
женат / замужем,12339,7.5%
в разводе,1195,7.1%
вдовец / вдова,959,6.6%


### Вывод

Клиенты которые состоят в браке лучше выплачивают кредит, что логично.  
Также стоит отметить, что выборка по клиентам "Не в браке" в 3 раза меньше, т.е при большей выборке процент может ещё увеличиться.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Да, действительно люди в браке лучше возвращают кредиты, но если мы оценим отдельные категории, то заметим, что наиболее рисковые группы это те люди, которые никогда не состояли в официально зарегистрированных отношениях

</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [70]:
(ks.groupby(['income_level'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,count,mean
income_level,,
Средний уровень дохода,5363,9.0%
Высокий уровень дохода,5363,8.5%
Низкий уровень дохода,5364,8.0%
Сверхвысокий уровень дохода,5364,7.0%


Клиенты с низким уровнем дохода платят лучше, чем со средним и высоким, что странно.  
Посмотрим подробнее.

In [71]:
(ks.groupby(['income_level','kids'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,,count,mean
income_level,kids,,
Средний уровень дохода,Есть дети,1824,10.1%
Высокий уровень дохода,Есть дети,1901,9.8%
Низкий уровень дохода,Есть дети,1722,9.6%
Средний уровень дохода,Нет детей,3539,8.5%
Высокий уровень дохода,Нет детей,3462,7.8%
Сверхвысокий уровень дохода,Есть дети,1916,7.4%
Низкий уровень дохода,Нет детей,3642,7.2%
Сверхвысокий уровень дохода,Нет детей,3448,6.7%


### Вывод

Как вариант можно предположить, что люди со средним и высоким уровнем дохода брали кредит на большую сумму(например ипотека)
и после рождения детей на время декрета одного из родителей из-за снижения уровня дохода имели просрочки по кредитам.  
Люди с низким уровнем дохода обычно более осмотрительно берут кредиты.  
Также люди с высоким уровнем дохода могут менее ответственно подходить к погашению кредита, так как не бояться процента за просрочку.  
В любом случае видна корреляция с наличием детей.  
Также стоит отметить, что есть очевидные проблемы с данными по доходам начиная с пропусков, заканчивая аномальными значениями.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Да, ты прав, хорошие выводы. Действительно люди среднего дохода и высокого хуже всех погашают кредит.

</div>

- Как разные цели кредита влияют на его возврат в срок?

In [72]:
(ks.groupby(['purpose_cat'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,count,mean
purpose_cat,,
автомобиль,4306,9.4%
образование,4013,9.2%
свадьба,2324,8.0%
недвижимость,6351,7.5%
жилье,4460,6.9%


In [73]:
(ks.groupby(['family_status','purpose_cat'])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

,,count,mean
family_status,purpose_cat,,
гражданский брак,образование,404,14.9%
не женат / не замужем,автомобиль,637,12.9%
гражданский брак,автомобиль,434,11.8%
не женат / не замужем,образование,577,10.7%
гражданский брак,недвижимость,586,9.4%
вдовец / вдова,автомобиль,218,9.2%
гражданский брак,жилье,403,8.9%
не женат / не замужем,недвижимость,937,8.6%
женат / замужем,автомобиль,2736,8.4%


### Вывод

Больше всего просрочек по кредитам на автомобиль 9.4% и образование 9.2%.  
Стоит уделить внимание клиентам из в гражданском браке и не женатым, которы берут кредиты на данные цели. 

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Да, действительно автомобиль и образование наиболее рисковые цели, тогда как недвижимость и свадьбы наоборот.

Молодец, что провел дополнительную группировку по семейному статусу!

</div>

### Шаг 4. Общий вывод

Перед тем как делать общий вывод посмотрим общую картину по всем категориям.  
Уровень дохода не берём, так как есть сомнения в его корректности.

In [74]:
(ks.groupby(['purpose_cat','kids','family_status_category',])['debt']
.agg(['count', 'mean'])
.sort_values('mean',ascending=False)
.style
.format({
    'mean': '{:,.1%}'.format,
})
.applymap(
     lambda x: 'background-color : red' if x>0.081 else '', 
     subset=['mean']))

## Вывод

- **8.1%** клиентов имели задолженность по возврату кредитов.  
- Есть зависимость между наличием детей, семейным положением, уровнем дохода и целей кредита на его возврат в срок, но при этом нет явной корелляции между показателями. В среднем разница около **4%**.
- Нужно обратить внимание на клиентов из категории **"Не в браке" и "Есть дети"**. Посмотреть динамику на более объёмной выборке.
- Из целей кредита повышенный процент просрочек в кредите на **автомобиль,образование и недвижимость**, особенно у клиентов с **детьми и в браке**.
- В среднем клиенты **без детей** реже имеют задолженность по кредитам, кроме категории **автокредитов**.

В качестве общей рекомендации заказчику следует обратить внимание на верификацию и андеррайтинг анкеты данных по заёмщикам.  
Есть большое колличество проблем в качестве данных, в частности в одной из ключевых метрик уровня дохода клиента.


<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА 2:</b> 

Молодец, получилась очень хорошая сводная таблица, мы можем много из нее узнать. Вывод тоже правильный, не смотря на явную безопасность отдельных целей просто по срезу целей, мы видим, что в совместном анализе они уже не выглядят настолько надежными. Необходимо учитывать набор факторов, так скажем их взаимодействие.

Именно такие задачи и решает машинное обучение! Если будет интересно углубись)
</div>